<a href="https://colab.research.google.com/github/aaravbmb/ML-speech-emotion-recognition-system/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Speech Emotion Recognition System

In [ ]:
# Importing Libraries

# General Libraries
import numpy as np
import pandas as pd
import os
import wave
import matplotlib.pyplot as plt

# Project-Specific Libraries
import librosa

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizer_v1 import rmsprop


In [ ]:
# Extract MFCC from WAV Audio File
def extract_mfcc(wav_file_name):
  y, sr = librosa.load(wav_file_name)
  mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
  return mfccs

In [ ]:
# Loading RADVESS Speech Data
radvess_speech_labels = []
radvess_speech_data = []
for dirname, _, filenmames in os.walk(''):
  for filename in filenmames:
    radvess_speech_labels.append(int(filename[7:8]) - 1)
    wav_file_name = os.path.join(dirname, filename)
    radvess_speech_data.append(extract_mfcc(wav_file_name))

In [ ]:
# Converting Data and Label to Array
radvess_speech_data_array = np.array(radvess_speech_data)
radvess_speech_labels_array = np.array(radvess_speech_labels)

# Making Categorical Labels
labels_categorical = to_categorical(radvess_speech_labels_array)

In [ ]:
# Splitting Data into Training and Testing Data
x_train, x_test, y_train, y_test = train_test_split(np.array(radvess_speech_data_array), labels_categorical, test_size=0.2, random_state=9)

In [ ]:
# Creating the LSTM Model
def create_model_LSTM():
  model = Sequential()
  model.add(LSTM(128, return_sequences=False, input_shape=(40,1)))
  model.add(Dense(64))
  model.add(Dropout(0.4))
  model.add(Activation('relu'))
  model.add(Dense(32))
  model.add(Dropout(0.4))
  model.add(Activation('relu'))
  model.add(dense(8))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
  return model


In [ ]:
# Training Process using LSTM Model
model_A = create_model_LSTM()
history = model_A.fit(np.expand_dims(radvess_speech_data_array[:training_samples],-1), labels_categorical[:training_samples], validation_data=(np.expand_dims(radvess_speech_data_array[training_samples:training_samples+validation_samples], -1), labels_categorical[training_samples:training_samples+validation_samples]), epochs=130, shuffle=True)

In [ ]:
# Loss Plot for LSTM Model

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Plots using LSTM Model

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Model Evaluation
model_A.evaluate(np.expand_dims(ravdess_)